In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, IntervalStrategy

torch.manual_seed(42)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-1.3b", bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = AutoModelForCausalLM.from_pretrained("EleutherAI/polyglot-ko-1.3b").cuda()
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(30005, 2048)

In [3]:
class MyDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer(f"""<|startoftext|>
            질문에 대답하세요.\n""" + txt + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

# 콜센터 데이터

In [4]:
import json
  
# Opening JSON file
f = open('./call_data/민원(콜센터) 질의응답_질병관리본부_약품식품_Training.json', encoding='UTF-8')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [5]:
text = ""
qna = []
for i in range(len(data)):
    if data[i]['고객질문(요청)']:
        text += "Q: " + data[i]['고객질문(요청)'] + "\n "
        if len(data[i + 1]['상담사답변']) > 10:
            text += "A: " + data[i + 1]['상담사답변'] 
            qna += [text]
    if data[i]['상담사질문(요청)']:
        text += "Q: " + data[i]['상담사질문(요청)'] + "\n "
        if len(data[i + 1]['고객답변']) > 10:
            text += "A: " + data[i + 1]['고객답변']
            qna += [text]
    text = ""

In [6]:
qna = qna[:200]

In [7]:
len(qna)

200

In [8]:
qna_series = pd.Series(qna)

In [9]:
max_length = max([len(tokenizer.encode(e)) for e in qna_series])
print("Max length: {}".format(max_length))

Max length: 52


In [10]:
dataset = MyDataset(qna_series, tokenizer, max_length = max_length)

In [11]:
import gc
import torch 
gc.collect() 
torch.cuda.empty_cache()

In [ ]:
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
training_args = TrainingArguments(output_dir='./results', num_train_epochs=5, logging_steps=1000,
                                  save_strategy=IntervalStrategy.NO,
                                  per_device_train_batch_size=2, per_device_eval_batch_size=2,
                                  warmup_steps=100, weight_decay=0.01, logging_dir='./logs')
Trainer(model=model, args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

C:\Users\User\anaconda3\envs\GPU3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


In [ ]:
generated = tokenizer("<|startoftext|>", return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(generated, do_sample=True, top_k=50, 
                                # bos_token='<|startoftext|>',
                                # eos_token='<|endoftext|>', pad_token='<|pad|>',
                                max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=5)

In [ ]:
for i, sample_output in enumerate(sample_outputs):
    print("{}: {} \n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

In [24]:
from huggingface_hub import notebook_login

notebook_login()

In [25]:
tokenizer.push_to_hub("eunyounglee/polyglot_kr_0330")

CommitInfo(commit_url='https://huggingface.co/eunyounglee/polyglot_kr_0330/commit/fb76ef1bbd321c77dca27c7e94ca758d0baab3a1', commit_message='Upload tokenizer', commit_description='', oid='fb76ef1bbd321c77dca27c7e94ca758d0baab3a1', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# model.push_to_hub("eunyounglee/polyglot_kr_0323")

Configuration saved in C:\Users\User\AppData\Local\Temp\tmpv5tcm9w5\config.json
Model weights saved in C:\Users\User\AppData\Local\Temp\tmpv5tcm9w5\pytorch_model.bin
Uploading the following files to eunyounglee/polyglot_kr_0323: config.json,pytorch_model.bin


In [27]:
torch.save(model.state_dict(), "./polyglot_ko_0330_2/pytorch_model.bin")

# QA

In [29]:
from transformers import pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0) 

In [38]:
qna[:10]

['Q: 코로나 검사결과는 언제 나와요? \n A: 6시간에서 하루 정도 소요됩니다. ',
 'Q: 결과는 어떻게 알 수 있어요? \n A: 문자로 양성, 음성 여부를 알려줍니다. . ',
 'Q: 하루가 지나도 연락이 안오면 어떻게 해요?\n A: 검사 받으신 진료소로 전화주셔서 확인하시면 됩니다. ',
 'Q: 친구 아버지가 돌아가셨는데 장례식장에 가도 돼요? \n A: 가셔도 되지만, 가급적 사람들이 많이 모이는 곳은 피하시는 것이 좋습니다. ',
 'Q: 장례식장에 마스크 안끼고 가도 돼요?\n A: 마스크는 꼭 착용하셔야 합니다. ',
 'Q: 장례식장에서 밥 먹을 수 있어요?\n A: 네, 한 방향으로 앉거나, 지그재그로 앉은 상태에서 식사 가능하십니다.',
 'Q: 장례식장에서도 거리두기 해야 돼요?\n A: 2m 거리두기를 원칙으로 하고 있습니다.',
 'Q: 열이 있는데 가도 될까요?\n A: 열이 있으시면 되도록 가지 않기를 권장합니다. ',
 'Q: 어떤 장소에서 의무적으로 마스크를 써야하나요?\n A: 버스,지하철과 같은 대중교통 이용할 떄와 병원 요양시설,집회등에서는 마스크를 의무적으로 착용핫야 합니다',
 'Q: 그럼 1단계 일땐 어떻게 적용되나요?\n A: 1단계 일땐 고위험 시설에서 의무적용됩니다']

In [54]:
prompt3 = "질문에 대답하세요.\nQ: 열이 있는데 장례식장에 가도 되나요?" #@param {type:"string"}
response_min_chars =  50#@param {type:"integer"}
response_max_chars =  100#@param {type:"integer"}

response_3 = generator(prompt3, do_sample=True, min_length=response_min_chars, 
                       max_length=response_max_chars,
                       clean_up_tokenization_spaces=True,
                       return_full_text=True)
out3_dict = response_3[0]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [55]:
out3_dict

{'generated_text': '질문에 대답하세요.\nQ: 열이 있는데 장례식장에 가도 되나요?\n A: 열이 있으시면 되도록 가지 않기를 권장합니다.   질문에 대답하세요.\nQ: 친척들과 노래방 갈 수 있어요? \n A: 비수도권 지역이라면 노래방 출입은 가능하십니다만 방역수칙을 잘 지키셔야됩니다.  '}

In [53]:
result = out3_dict['generated_text'].split("\n")
print(result[1], result[2])

Q: 코로나 검사결과는 언제 나와?  A: 6시간에서 하루 정도 소요됩니다.  시에서는 마스크 구매할때도 방역수칙을 잘 지키셔야됩니다 http://pf.kakao.com/_JCMKR/chat이르면 추석 전 사우나 갈 수 있는거


In [33]:
from happytransformer import HappyGeneration

happy_gen = HappyGeneration(model_type = "GPT-NEO", model_name = "eunyounglee/polyglot_kr_0330")

Downloading:   0%|          | 0.00/618 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.43G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213 [00:00<?, ?B/s]

03/30/2023 15:42:33 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [34]:
# result = happy_gen.generate_text("계량기에서 소리가 나는데 교환이 가능할까요?")

# print(result.text)

In [87]:
from happytransformer import HappyGeneration, GENSettings

#---------------------------------------------------

greedy_settings = GENSettings(max_length=100)
output_greedy = happy_gen.generate_text(
    "코로나에 걸리면 죽나요?",
    args=greedy_settings)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [88]:
result = output_greedy.text

In [89]:
result.replace("\n", "")[1:]

'코로나19에 걸린다고 해서 모두 폐가 망가지는 것은 아니며 그렇게 심하지않은 것으로 나타났습니다. '

## 질문 예시

고객번호란 무엇이고, 어떻게 확인할 수 있나요?

○ 고객번호는 전기사용 계약단위별로 부여되는 고유번호입니다. 전기요금 청구서 우측 상단에서 확인하실 수 있으며 이를 통해 사이버지점에서 전기 사용 관련 유용한 정보를 제공받으실 수 있습니다.○ 만약 고객님의 전기요금이 아파트 관리비 청구서에 포함되어 청구된다면 아파트 전체를 하나의 고객번호로 한전과 계약한 경우로 이때, 세대별로 부여되는 고객번호는 없습니다. 다만, 대가족요금, 복지할인요금 등은 아파트 전체의 고객번호로 신청이 가능하오니 아파트 관리사무소나 한전 고객센터(국번없이 123)로 아파트 고객번호를 문의하시기 바랍니다.

In [33]:
greedy_settings = GENSettings(max_length=200)
output_greedy = happy_gen.generate_text(
    "고객번호가 무엇인가요?",
    args=greedy_settings)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [34]:
output_greedy.text.replace("\n", "")

'전기사용계약단위별로 부여되는 고유번호입니다. 전기요금 청구서 우측 상단에서 확인하실 수 있으며 이를 통해 사이버지점에서 전기 사용 관련 유용한 정보를 제공받으실 수 있습니다.'

변전소는 무엇이며 무슨 기능을 하는 곳입니까?

변전소라함은 구외로부터 보내오는 66kV, 154kV, 345kV 등의 송전전압을 구내에 시설한 변압기등에 의하여 전압을 높이거나 낮추어 다시 구외로 보내는 장소로서 한전이 소유하는 것을 말합니다.전압을 변환하는 방법은 전자유도작용의 원리를 이용하여 만든 변압기를 사용하는데 변압기에는철심에 코일을 감아서 만든 것입니다. 또한 변전소는 변압기 이외에 차단기, 단로기, 모선 등으로구성되어 있습니다. 그러나 전압을 낮춰서 자기의 공장 또는 건물내에만 쓰는 경우는 변전소라하지 않고 수전실, 변전실 등의 명칭을 쓰고 있습니다.

전기요금을 미리 계산해 볼 수 있나요.	`전기요금계산기(새창열림)`href=`http://cyber.kepco.co.kr/ckepco/front/jsp/CY/J/A/CYJAPP000.jsp`>사이버지점전기요금계산기에서 가능합니다.


In [52]:
greedy_settings = GENSettings(max_length=300)
output_greedy = happy_gen.generate_text(
    "전기요금을 미리 계산해 보고 싶습니다.",
    args=greedy_settings)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [54]:
output_greedy.text.replace("\n", "")

'`전기요금계산기(새창열림)`href=`http://cyber.kepco.co.kr/ckepco/front/jsp/CY/F/A/CYFAPP0016001.jsp`>사이버지점전기요금계산기에서 가능합니다.'

전기요금을 미납한 경우 연체료율은 몇%입니까?

전기요금을 납기일까지 납부하지 아니한 경우에는 연체료를 부담하셔야 합니다. 연체료율은 기간별로 차등적용하며 일수계산하여 부과하고 있습니다. - 전기요금 연체료 : 처음 1개월 미납시 1.5%로 일수계산,다음 1개월 미납시 1.5%로 일수계산 - TV수신료에대한 연체료는 미납월과 관계없이 5% (주한외국군 부대, 주한외교기관 고객에 대하여는 연체료를 적용하지 않음)청구서 배달 지연 등 아래의 경우에는납기일을 다시 지정하여 요금청구서를 재발행하여 드립니다. 다시 지정된 납기일이란 청구서를 재발급하여 고객에게 드린 날로부터 7일째 되는 날을 기준으로 정하는 것을 원칙으로 하고 있습니다.1.한전의 귀책에 의하여 송달이 지연된 경우,2. 우편물 폭주로 인하여 송달이 지연된 경우,3.기타 한전에서송달 지연을 인정하는 경우※ 연체료 면제 또는 납기일 연장된 고객이 다시 지정한 납기일까지 미납시에는 다음달 요금에 연체료가 합산 부과됨.※ 전기요금을 납기일로부터 2개월이 되는 날까지 납부하지 않는 고객에 대하여는 단전일(해지예정일) 7일전까지 미납요금 납부에 관한 사항을 안내 후 전기사용계약을 해지하므로 전기요금을 체납하지 않도록 주의하시기 바랍니다.

In [46]:
greedy_settings = GENSettings(max_length=350)
output_greedy = happy_gen.generate_text(
    "전기요금을 미납한 경우 연체료율은 몇%입니까?",
    args=greedy_settings)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [47]:
output_greedy.text.replace("\n", "")

'전기요금을 납기일까지 납부하지 아니한 경우에는 연체료를 부담하셔야 합니다. 연체료율은 기간별로 차등적용하며 일수계산하여 부과하고 있습니다. - 전기요금 연체료 : 처음 1개월 미납시 1.5%로 일수계산,다음 1개월 미납시 1.5%로 일수계산 - TV수신료에대한 연체료는 미납월과 관계없이 5% (주한외국군 부대, 주한외교기관 고객에 대하여는 연체료를 적용하지 않음)청구서 배달 지연 등 아래의 경우에는납기일을 다시 지정하여 요금청구서를 재발행하여 드립니다. 다시 지정된 납기일이란 청구서를 재발급하여 고객에게 드린 날로부터 7일째 되는 날을 기준으로 정하는 것을 원칙으로 하고 있습니다.1.한전의 귀책에 의하여 송달이 지연된 경우,2. 우편물 폭주로 인하여 송달이 지연된 경우,3.기타 한전에서송달 지연을 인정하는 경우※ 연체료 면제 또는 납기일 연장된 고객이 다시 지정한 납기일까지 미납시에는 다음달 요금에 연체료가 합산 부과됨.※ 전기요금을 납기일로부터 2개월이 되는 날까지 납부하지 않는 고객에 대하여는 단전일(해지예정일) 7일전까지 미납요금 납부에 관한 사항을 안내 후 전기사용계약을 해지하므로 전기요금을 체납하지 않도록 주의하시기 바랍니다.'

In [ ]:
greedy_settings = GENSettings(max_length=350)
output_greedy = happy_gen.generate_text(
    "전기요금을 미납한 경우 연체료율은 몇%입니까?",
    args=greedy_settings)

# 정확성 판단

In [ ]:
greedy_settings = GENSettings(max_length=700)

In [62]:
!pip install nltk

     ---------------------------------------- 1.5/1.5 MB 48.4 MB/s eta 0:00:00
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)


In [82]:
short_elec = []
for i in range(len(elec)):
    if len(elec.loc[i, '내용']) <= 700:
        short_elec.append([elec.loc[i, '제목'], elec.loc[i, '내용']])

In [ ]:
import nltk
result = []
BLEUscore_sum = []
for i in range(len(short_elec)): # len(elec)):
    title = short_elec[i][0].replace(".", "?")
    output_greedy = happy_gen.generate_text(title, args=greedy_settings)
    res = output_greedy.text.replace("\n", "")
    BLEU = nltk.translate.bleu_score.sentence_bleu([short_elec[i][1]], res)  # BLEU 스코어 측정
    BLEUscore_sum.append(BLEU) 
    result.append(res)

In [88]:
sum(BLEUscore_sum) / len(short_elec)

0.7881187745869855

In [90]:
lis = []
for i in range(len(short_elec)):
    l = [short_elec[i][0], short_elec[i][1], result[i], BLEUscore_sum[i]]
    lis.append(l)

In [91]:
elec_df = pd.DataFrame(lis, columns=['질문', '답변', '예측답변', 'BLEU'])

In [95]:
elec_df.to_csv("elec_result.csv", encoding='utf-8-sig')

# 평문으로 학습하기 

In [1]:
with open('esg.txt', encoding='utf-8') as f:
    lines = f.readlines()

In [2]:
import pandas as pd 
df = pd.Series(lines)

In [3]:
# \n 없애기 
df = [i for i in df if i != '\n']

In [4]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, IntervalStrategy

torch.manual_seed(42)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b", bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = AutoModelForCausalLM.from_pretrained("EleutherAI/polyglot-ko-5.8b").cuda()
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/164 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/459 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.9G [00:00<?, ?B/s]

Embedding(30005, 4096)

In [9]:
class NetflixDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [10]:
import pandas as pd
sr = pd.Series(df)

In [11]:
max_length = 0
for i in range(len(sr)):
    max_length = max(max_length, len(sr[i]))

In [12]:
dataset = NetflixDataset(df, tokenizer, max_length=max_length)

In [13]:
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
training_args = TrainingArguments(output_dir='./results', num_train_epochs=5, logging_steps=5000,
                                  save_strategy=IntervalStrategy.NO,
                                  per_device_train_batch_size=2, per_device_eval_batch_size=2,
                                  warmup_steps=100, weight_decay=0.01, logging_dir='./logs')
Trainer(model=model, args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

C:\Users\User\anaconda3\envs\gpt\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 72
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 180
  Number of trainable parameters = 5884444672


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.00 MiB (GPU 0; 23.99 GiB total capacity; 23.14 GiB already allocated; 0 bytes free; 23.21 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [13]:
torch.save(model.state_dict(), "./polyglot_ko_0324/pytorch_model.bin")

In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [16]:
tokenizer.push_to_hub("eunyounglee/polyglot_kr_0324")

tokenizer config file saved in C:\Users\User\AppData\Local\Temp\tmp415kk3f0\tokenizer_config.json
Special tokens file saved in C:\Users\User\AppData\Local\Temp\tmp415kk3f0\special_tokens_map.json
Uploading the following files to eunyounglee/polyglot_kr_0324: special_tokens_map.json,tokenizer.json,tokenizer_config.json


CommitInfo(commit_url='https://huggingface.co/eunyounglee/polyglot_kr_0324/commit/b46f9b2cec65c6371b05a27e427df269ea30d9ad', commit_message='Upload tokenizer', commit_description='', oid='b46f9b2cec65c6371b05a27e427df269ea30d9ad', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
sr

0       기업들 대다수가 올해 ESG 경영 규모를 작년 수준 이상으로 유지하겠다고 밝혔다.\n
1     전국경제인연합회(회장 허창수)가 여론조사기관 모노리서치에 의뢰해 매출액 500대 기...
2     전경련은 올해 ESG 사업 규모 확장의 배경을 기후 위기 극복을 위한 글로벌 공감대...
3     E·S·G 중 가장 중요한 이슈로는 E(환경)가 82.0%로 가장 높았으며 S(사회...
4     기업들은 NDC 2030 온실가스 감축 목표 달성을 위한 기후변화 대응전략과 구체적...
                            ...                        
75    앞서, 본사는 2021년 제주 고유의 맛과 특색을 살린 제주 특화 브랜드 ‘제주담음...
76    푸드테크기업 힘난다에서는 본사에서 운영하는 수제버거 프랜차이즈 힘난다버거를 통해 프...
77    힘난다는 최근 공식 블로그를 통해 '2023 힘나는 약속'을 선보여 가맹점에 대한 ...
78    본사 관계자는 "외식업계의 불황기인 만큼, 본사의 존재 이유는 전국 가맹점을 소생하...
79    업계 관계자는 "100년 기업으로 성장하기 위해서는 재무적인 부분만을 충족함이 아니...
Length: 80, dtype: object

In [20]:
from happytransformer import HappyGeneration, GENSettings

#---------------------------------------------------
happy_gen = HappyGeneration(model_type="GPT-NEO", model_name="eunyounglee/polyglot_kr_0324")
greedy_settings = GENSettings(max_length=300)
output_greedy = happy_gen.generate_text(
    "전국경제인연합회의 회장은 누군가요?",
    args=greedy_settings)


Downloading:   0%|          | 0.00/618 [00:00<?, ?B/s]

loading configuration file config.json from cache at C:\Users\User/.cache\huggingface\hub\models--eunyounglee--polyglot_kr_0324\snapshots\c5ce5b1a99c93283c83bdbab0dec500a04029442\config.json
Model config GPTNeoXConfig {
  "_name_or_path": "eunyounglee/polyglot_kr_0324",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "bos_token_id": 0,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "rotary_emb_base": 10000,
  "rotary_pct": 0.5,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use_cache": true,
  "use_parallel_residual": true,
  "vocab_size": 30005
}



Downloading:   0%|          | 0.00/5.43G [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at C:\Users\User/.cache\huggingface\hub\models--eunyounglee--polyglot_kr_0324\snapshots\c5ce5b1a99c93283c83bdbab0dec500a04029442\pytorch_model.bin
All model checkpoint weights were used when initializing GPTNeoXForCausalLM.

All the weights of GPTNeoXForCausalLM were initialized from the model checkpoint at eunyounglee/polyglot_kr_0324.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPTNeoXForCausalLM for predictions without further training.


Downloading:   0%|          | 0.00/381 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213 [00:00<?, ?B/s]

loading file tokenizer.json from cache at C:\Users\User/.cache\huggingface\hub\models--eunyounglee--polyglot_kr_0324\snapshots\c5ce5b1a99c93283c83bdbab0dec500a04029442\tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\User/.cache\huggingface\hub\models--eunyounglee--polyglot_kr_0324\snapshots\c5ce5b1a99c93283c83bdbab0dec500a04029442\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\User/.cache\huggingface\hub\models--eunyounglee--polyglot_kr_0324\snapshots\c5ce5b1a99c93283c83bdbab0dec500a04029442\tokenizer_config.json
03/24/2023 16:18:08 - INFO - happytransformer.happy_transformer -   Using model: cuda
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [22]:
sr[12]

'롯데건설은 산업통상자원부 한국에너지기술평가원의 ‘시멘트 산업 발생 이산화탄소 활용 탄산화 기술개발’ 연구와 한국산업기술평가관리원의 ‘이산화탄소 반응 경화 시멘트 개발’ 연구에 공동 연구사로 참여 중이다. 탄산화 기술개발 연구는 산업 공정에서 발생한 이산화탄소를 고농도로 포집해 레미콘 공장으로 운송 후 레미콘 생산에 이용하는 기술 개발을 목적으로 한다. 시멘트 개발 연구는 이산화탄소와 반응해 굳어지는 시멘트를 개발한다.\n'

In [21]:
output_greedy.text

' 기업들의 대표이사와 이사회 의장이 분리된 곳은 어디인가요.\n 등을 중심으로 한 ESG 경영이 필수가 된 가운데, 기업들의 ESG 경영에 대한 인식과 실행력에서는 보완할 부분이 적지 않다는 평가다. 특히 매출이 높은 일부 기업 중에는 외형에 비해 ESG 경영을 소비자 친화 정책의 일환으로 접근하는 경우도 있다. 관점의 전환이 필요하다는 지적이다.\n'

In [27]:
output_greedy = happy_gen.generate_text(
    "ESG가 무엇인가요?",
    args=greedy_settings)
output_greedy.text

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


' 기업들은 어떤 형태로 ESG 경영을 실천하고 있나요. 사례를 통해 ESG의 실천이란 어떤 것인지 구체적으로 알아봅니다. \n'